In [93]:
import math
import numpy as np
import pandas as pd
from collections import Counter
from PyImpetus import PPIMBR
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
import sklearn
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import time

In [94]:
df = pd.read_csv('data/only2_big_with_sf.csv')
# df = df.transpose()
df.set_index('cell_line', inplace=True)
# df = df.transpose()
df.head()

,7105,8813,57147,55732,2268,3075,2519,2729,4800,90529,...,8336,84311,126661,79447,283521,29122,9997,388963,6905,SF
cell_line,,,,,,,,,,,,,,,,,,,,,
NCIH1568-2,5.626439,6.855242,2.185867,4.079805,0.084064,0.176323,6.824513,4.956521,3.766595,2.298658,...,0.622930,5.373996,1.956057,5.482203,0.422233,0.176323,0.918386,1.778209,4.050502,0.607940
SCC9-2,3.964399,5.718088,2.260026,3.451541,0.084064,0.782409,6.039796,5.778734,4.903038,2.711495,...,1.536053,5.042207,2.344828,3.937344,0.000000,0.042644,2.475085,1.176323,4.174726,0.573035
HCC1428-2,4.002703,5.870365,3.621759,3.861955,0.014355,0.485427,4.614121,5.286881,4.429616,3.442280,...,3.033863,4.539159,1.970854,5.895303,0.622930,0.014355,0.918386,2.244887,5.555816,0.450188
U2OS-2,5.098453,6.543496,2.117695,4.115200,0.014355,0.432959,5.684258,4.847997,5.108524,2.189034,...,0.176323,5.028569,2.967169,4.995937,0.111031,0.042644,2.010780,1.925999,4.635754,0.526517
LU99-2,3.665620,6.716717,2.100978,4.176323,0.000000,1.263034,6.124948,4.395063,5.070389,2.669027,...,1.321928,6.613679,1.438293,4.349082,0.000000,0.111031,1.111031,1.632268,5.197315,0.625161


In [95]:
# df = df.drop(["cell_line"], axis=1)
display(df.head())
data, Y = df.drop(["SF"], axis=1), df["SF"].values
# Lets check out the shape of our data
print("Data shape: ", data.shape, "Target Variable shape: ", Y.shape)

,7105,8813,57147,55732,2268,3075,2519,2729,4800,90529,...,8336,84311,126661,79447,283521,29122,9997,388963,6905,SF
cell_line,,,,,,,,,,,,,,,,,,,,,
NCIH1568-2,5.626439,6.855242,2.185867,4.079805,0.084064,0.176323,6.824513,4.956521,3.766595,2.298658,...,0.622930,5.373996,1.956057,5.482203,0.422233,0.176323,0.918386,1.778209,4.050502,0.607940
SCC9-2,3.964399,5.718088,2.260026,3.451541,0.084064,0.782409,6.039796,5.778734,4.903038,2.711495,...,1.536053,5.042207,2.344828,3.937344,0.000000,0.042644,2.475085,1.176323,4.174726,0.573035
HCC1428-2,4.002703,5.870365,3.621759,3.861955,0.014355,0.485427,4.614121,5.286881,4.429616,3.442280,...,3.033863,4.539159,1.970854,5.895303,0.622930,0.014355,0.918386,2.244887,5.555816,0.450188
U2OS-2,5.098453,6.543496,2.117695,4.115200,0.014355,0.432959,5.684258,4.847997,5.108524,2.189034,...,0.176323,5.028569,2.967169,4.995937,0.111031,0.042644,2.010780,1.925999,4.635754,0.526517
LU99-2,3.665620,6.716717,2.100978,4.176323,0.000000,1.263034,6.124948,4.395063,5.070389,2.669027,...,1.321928,6.613679,1.438293,4.349082,0.000000,0.111031,1.111031,1.632268,5.197315,0.625161


Data shape:  (525, 15139) Target Variable shape:  (525,)


In [96]:
# # Use Train-test split for understanding the performance of PyImpetus
# x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=27, shuffle=True)
# # We want to time our algorithm
# start = time.time()

In [97]:
# # Convert the data into numpy arrays
# x_train, x_test = x_train.values, x_test.values
#
# model = DecisionTreeRegressor(random_state=27)
# model.fit(x_train, y_train)
# preds = model.predict(x_test)
# score = mean_squared_error(y_test, preds)
# print("MSE: ", score)
# end = time.time()
# print("Time Required (in seconds): ", end-start)

In [99]:
# Use Train-test split for understanding the performance of PyImpetus
x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=0.1, random_state=27, shuffle=True)
# We want to time our algorithm
start = time.time()

# Create a PyImpetus classification object and initialize with required parameters
model = PPIMBR(model=sklearn.linear_model.BayesianRidge(), cv=3, p_val_thresh=0.001, num_simul=20, simul_size=0.2, sig_test_type="non-parametric", random_state=27, verbose=2)
# Fit this above object on the train part and transform the train dataset into selected feature subset
# NOTE: x_train has to be a dataframe and y_train has to be a numpy array
x_train = model.fit_transform(x_train, y_train)
# Transform the test set as well
# NOTE: x_test has to be a dataframe
x_test = model.transform(x_test)
# Check out the number of features selected
print("Number of features selected: ", len(model.MB))
print(model.MB)
# Convert the data into numpy arrays
x_train, x_test = x_train.values, x_test.values

model = DecisionTreeRegressor(random_state=27)
model.fit(x_train, y_train)
preds = model.predict(x_test)
score = mean_squared_error(y_test, preds)
print("Score: ", score)
end = time.time()
print("Time Required (in seconds): ", end-start)
model.feature_importance()

Progress bar: 100%|██████████| 3/3 [05:27<00:00, 109.10s/it]

Number of features selected:  13
['1462', '200916', '254158', '54442', '54851', '1876', '55175', '2826', '64714', '1995', '132946', '10921', '9294']
Score:  0.06258900337859535
Time Required (in seconds):  327.3015339374542


AttributeError: 'DecisionTreeRegressor' object has no attribute 'feature_importance'